In [1]:
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter
import numpy as np
from utils import parse_json_logs_to_pd, get_summary_stats, get_counts, get_sums, get_ts, hist, compute_entropy, bar, ts_line

In [2]:
# todo: add unused plot
# todo: fix ts

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
target_logs = "ex_3"
logs_location = f"./logs/{target_logs}"

In [5]:
conn_df = parse_json_logs_to_pd(f"{logs_location}/conn.log")
dns_df = parse_json_logs_to_pd(f"{logs_location}/dns.log")
http_df = parse_json_logs_to_pd(f"{logs_location}/http.log")

In [6]:
conn_df['dt'] = pd.to_datetime(conn_df['ts'], unit='s')
dns_df['dt'] = pd.to_datetime(dns_df['ts'], unit='s')
http_df['dt'] = pd.to_datetime(http_df['ts'], unit='s')

In [7]:
conn_df.dt.max()

Timestamp('2024-02-17 16:56:00.358997107')

In [8]:
conn_df.dt.min()

Timestamp('2024-02-17 16:37:13.418903112')

# conn.log

In [9]:
conn_df

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,dt
0,1.708188e+09,Cja8aA3CnwXXSdIMMc,192.168.220.43,62358,52.36.54.134,443,tcp,ssl,0.259585,728.0,5555.0,SF,True,False,0,ShADadFf,16,1568,13,6239,2024-02-17 16:37:19.878324032
1,1.708188e+09,CfQq6P2pf89M7w3jc1,192.168.220.132,65493,20.189.173.13,443,tcp,ssl,0.209493,4737.0,5042.0,SF,True,False,0,ShADdaFf,15,5349,14,5614,2024-02-17 16:37:20.012911081
2,1.708188e+09,Cu2LQfRd6gK0afVx1,10.128.1.29,55240,10.160.40.248,443,tcp,ssl,0.088309,967.0,2598.0,SF,True,True,0,ShADadfF,8,1299,8,2930,2024-02-17 16:37:20.808995962
3,1.708188e+09,CBi3Bk2MN9YQwtt0pl,192.168.220.2,50973,192.112.36.4,53,udp,dns,NaN,NaN,NaN,S0,True,False,0,D,1,56,0,0,2024-02-17 16:37:16.442317009
4,1.708188e+09,CG14V02FHsSkSA59q2,192.168.220.2,2144,192.58.128.30,53,udp,dns,NaN,NaN,NaN,S0,True,False,0,D,1,56,0,0,2024-02-17 16:37:16.444914103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24264,1.708188e+09,C5b6El1LLKkOYp4wIa,192.168.220.69,49989,40.83.240.146,443,tcp,NaN,980.171048,1.0,0.0,OTH,True,False,0,DaT,5,205,5,260,2024-02-17 16:39:33.620764971
24265,1.708188e+09,C0N6302hCjVM2lTEDk,10.128.1.29,54939,10.128.0.202,22,tcp,NaN,1126.614349,892376.0,9183680.0,OTH,True,True,0,DdAa,25010,1892776,26174,10230640,2024-02-17 16:37:13.418903112
24266,1.708188e+09,Cse1hp2HEayMGLeFMh,192.168.220.69,50028,40.83.240.146,443,tcp,NaN,955.103849,104.0,173.0,OTH,True,False,0,DadAT,6,347,5,421,2024-02-17 16:38:37.574867010
24267,1.708189e+09,C0XBbH2WJc4pQjqb45,192.168.220.73,45898,192.168.220.121,389,tcp,"ldap_tcp,ssl",0.042461,968.0,5464.0,RSTR,True,True,0,ShADadFrR,16,1796,11,6044,2024-02-17 16:56:00.164738894


In [10]:
get_counts(conn_df, "id.orig_h", time_slot="1min")

,id.orig_h,time_slot,Count
87,10.128.1.151,2024-02-17 16:52:00,1500
86,10.128.1.151,2024-02-17 16:51:00,1445
85,10.128.1.151,2024-02-17 16:50:00,1441
84,10.128.1.151,2024-02-17 16:49:00,1429
117,10.166.245.91,2024-02-17 16:49:00,1391
...,...,...,...
434,fe80::be24:11ff:fe39:72b9,2024-02-17 16:54:00,1
435,fe80::cb6f:2434:ebe3:56ee,2024-02-17 16:54:00,1
0,0.0.0.0,2024-02-17 16:54:00,1
437,fe80::ce3b:e1de:cb30:9a7e,2024-02-17 16:47:00,1


In [11]:
_df = get_counts(conn_df, ["id.orig_h", "conn_state"])

In [12]:
get_counts(conn_df, ["conn_state", "id.orig_h"], time_slot="1min")

,conn_state,id.orig_h,time_slot,Count
186,REJ,10.128.1.151,2024-02-17 16:40:00,973
362,S0,10.128.1.151,2024-02-17 16:53:00,892
361,S0,10.128.1.151,2024-02-17 16:52:00,751
358,S0,10.128.1.151,2024-02-17 16:49:00,727
359,S0,10.128.1.151,2024-02-17 16:50:00,721
...,...,...,...,...
32,OTH,10.128.0.202,2024-02-17 16:51:00,1
31,OTH,10.128.0.202,2024-02-17 16:46:00,1
30,OTH,10.128.0.202,2024-02-17 16:41:00,1
29,OTH,10.100.123.2,2024-02-17 16:54:00,1


In [13]:
failed_conns = conn_df[conn_df.conn_state.isin(["S0", "REJ"])]

In [14]:
get_counts(conn_df, "id.orig_h").head(20)

,id.orig_h,Count
26,10.128.1.151,8609
44,192.168.220.111,1778
31,10.129.248.21,1422
36,10.166.245.91,1392
33,10.158.115.188,1379
35,10.162.244.85,1377
51,192.168.220.53,807
34,10.159.36.161,717
38,10.180.105.140,714
32,10.135.237.219,669


In [15]:
get_counts(conn_df, "id.orig_h", time_slot="1min").head(20)

,id.orig_h,time_slot,Count
87,10.128.1.151,2024-02-17 16:52:00,1500
86,10.128.1.151,2024-02-17 16:51:00,1445
85,10.128.1.151,2024-02-17 16:50:00,1441
84,10.128.1.151,2024-02-17 16:49:00,1429
117,10.166.245.91,2024-02-17 16:49:00,1391
88,10.128.1.151,2024-02-17 16:53:00,1276
82,10.128.1.151,2024-02-17 16:40:00,1002
115,10.162.244.85,2024-02-17 16:53:00,849
112,10.158.115.188,2024-02-17 16:52:00,844
106,10.129.248.21,2024-02-17 16:51:00,791


In [16]:
get_counts(conn_df, ["id.orig_h", "id.resp_h"], time_slot="1min").head(20)

,id.orig_h,id.resp_h,time_slot,Count
239,10.128.1.151,192.168.220.30,2024-02-17 16:40:00,1002
254,10.128.1.151,192.168.220.53,2024-02-17 16:53:00,551
262,10.128.1.151,192.168.220.69,2024-02-17 16:53:00,455
253,10.128.1.151,192.168.220.53,2024-02-17 16:52:00,449
259,10.128.1.151,192.168.220.69,2024-02-17 16:50:00,432
260,10.128.1.151,192.168.220.69,2024-02-17 16:51:00,406
258,10.128.1.151,192.168.220.69,2024-02-17 16:49:00,394
250,10.128.1.151,192.168.220.53,2024-02-17 16:49:00,391
252,10.128.1.151,192.168.220.53,2024-02-17 16:51:00,376
261,10.128.1.151,192.168.220.69,2024-02-17 16:52:00,364


In [17]:
hist(conn_df, "duration")

In [18]:
short_conns = conn_df[conn_df["duration"] < 10]
hist(short_conns, "duration")

In [19]:
very_short_conns = conn_df[conn_df["duration"] < 1]
hist(very_short_conns, "duration")

In [20]:
get_counts(very_short_conns, ["id.orig_h", "conn_state"])

,id.orig_h,conn_state,Count
22,10.128.1.151,REJ,3934
88,192.168.220.111,RSTR,1041
113,192.168.220.53,SF,737
70,10.166.245.91,SF,636
65,10.162.244.85,SF,626
...,...,...,...
189,fe80::8bd:e660:f33e:f359,OTH,1
192,fe80::ac9e:707:c970:9942,OTH,1
193,fe80::be24:11ff:fe39:72b9,OTH,1
194,fe80::cb6f:2434:ebe3:56ee,OTH,1


In [21]:
avg_dur = get_summary_stats(very_short_conns, ["id.orig_h"], "duration", ascending=True)
avg_dur[avg_dur["conn_count"] > 10]

,id.orig_h,duration_mean,duration_std,conn_count
91,fe80::6a4f:64ff:fe4d:22cc,0.000413,0.000335,14
38,192.168.220.66,0.003469,0.026949,559
41,192.168.220.84,0.004231,0.016090,512
31,192.168.220.111,0.011735,0.040548,1560
1,10.0.0.50,0.012360,0.041265,14
17,10.128.1.151,0.014574,0.046103,4474
35,192.168.220.30,0.015441,0.050463,309
33,192.168.220.132,0.021714,0.084570,480
20,10.128.59.8,0.033940,0.024400,25
34,192.168.220.241,0.038462,0.100219,122


In [22]:
get_counts(conn_df[conn_df["id.orig_h"] == "10.128.1.151"], ["id.resp_h", "conn_state"])

,id.resp_h,conn_state,Count
19,192.168.220.69,S0,1994
15,192.168.220.53,S0,1982
7,192.168.220.30,REJ,1961
21,192.168.220.84,REJ,990
0,192.168.220.121,REJ,985
17,192.168.220.69,RSTO,175
14,192.168.220.53,RSTO,173
16,192.168.220.53,SF,78
5,192.168.220.121,SF,68
12,192.168.220.30,SF,35


In [23]:
long_conns = conn_df[conn_df["duration"] > 60]
hist(long_conns, "duration")

In [24]:
hist(conn_df, "orig_bytes")

In [25]:
get_summary_stats(conn_df[conn_df["orig_bytes"] > 0], "id.resp_h", "orig_bytes").sort_values("orig_bytes_std", ascending=True).head(20)

,id.resp_h,orig_bytes_mean,orig_bytes_std,conn_count
299,184.23.23.68,1.0,0.0,3
461,ff02::2,16.0,0.0,30
371,50.205.57.38,48.0,0.0,4
340,216.31.17.12,48.0,0.0,2
126,10.100.112.222,37.0,0.0,7
148,10.100.114.222,37.0,0.0,8
214,10.100.120.222,37.0,0.0,8
247,10.100.123.222,37.0,0.0,5
159,10.100.115.222,37.0,0.0,8
16,10.100.102.222,37.0,0.0,7


In [26]:
get_summary_stats(conn_df[conn_df["resp_bytes"] > 0], "id.resp_h", "resp_bytes").sort_values("resp_bytes_std", ascending=True).head(20)

,id.resp_h,resp_bytes_mean,resp_bytes_std,conn_count
411,fe80::6a4f:64ff:fe4d:22cc,16.0,0.0,14
346,45.32.207.136,48.0,0.0,2
352,45.79.51.42,48.0,0.0,4
377,69.164.213.136,48.0,0.0,5
356,50.205.57.38,48.0,0.0,4
371,64.79.100.196,48.0,0.0,13
355,5.78.89.3,48.0,0.0,3
281,162.159.200.1,48.0,0.0,2
192,10.100.118.222,1053.0,0.0,8
307,198.71.50.75,48.0,0.0,2


In [27]:
large_or_b = conn_df[conn_df["orig_bytes"] > 999999]

In [28]:
large_or_b

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,dt,time_slot
3959,1.708188e+09,C0lP2CwIJDM1j3JLh,10.135.237.219,35066,192.168.220.30,80,tcp,http,0.216738,515701896.0,1.836993e+09,SF,True,True,515701707,SADhadGfFtfF,11,770,8,6420,2024-02-17 16:44:01.369543076,2024-02-17 16:44:00


In [29]:
hist(conn_df[conn_df["orig_bytes"] < 999999], "orig_bytes")

In [30]:
cum_or_b = get_sums(conn_df, "id.orig_h", "orig_bytes")

In [31]:
hist(cum_or_b, "orig_bytes")

In [32]:
large_cum_or_b = cum_or_b[cum_or_b["orig_bytes"] > 999999]

In [33]:
large_cum_or_b

,id.orig_h,orig_bytes,conn_count
32,10.135.237.219,516079216.0,669
51,192.168.220.53,2764186.0,807
44,192.168.220.111,1599383.0,1778


In [34]:
hist(conn_df, "resp_bytes")

In [35]:
large_re_b = conn_df[conn_df["resp_bytes"] > 999999]
large_re_b

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,dt,time_slot
3959,1.708188e+09,C0lP2CwIJDM1j3JLh,10.135.237.219,35066,192.168.220.30,80,tcp,http,0.216738,515701896.0,1.836993e+09,SF,True,True,515701707,SADhadGfFtfF,11,770,8,6420,2024-02-17 16:44:01.369543076,2024-02-17 16:44:00
4093,1.708188e+09,CogtyY2UJKrZQT9UO4,192.168.220.73,57466,151.101.66.49,443,tcp,ssl,0.244002,808.0,1.038397e+06,SF,True,False,0,ShADadFfR,216,12024,737,1076729,2024-02-17 16:44:02.027178049,2024-02-17 16:44:00
7671,1.708189e+09,CYRRBk1qFDWgidLC8g,10.166.245.91,48586,10.100.120.73,993,tcp,ssl,3.552884,14525.0,1.903329e+06,SF,True,True,344190,ShADadggtWfFR,223,26165,1413,1979957,2024-02-17 16:49:13.783803940,2024-02-17 16:49:00
7672,1.708189e+09,CSyX5E4ybRrBQNFzd9,10.166.245.91,48586,192.168.220.73,993,tcp,ssl,3.552701,14525.0,1.903329e+06,SF,True,True,0,ShADadtWfFR,223,26165,345,1924421,2024-02-17 16:49:13.784280062,2024-02-17 16:49:00
7674,1.708189e+09,CnfExw1BOpRJw9pYJ3,10.166.245.91,51330,10.100.107.73,993,tcp,ssl,3.104701,14510.0,1.903143e+06,SF,True,True,372637,ShADadggtWfFR,278,29022,1460,1983399,2024-02-17 16:49:14.314500093,2024-02-17 16:49:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24171,1.708189e+09,C8LxAc3KPHC4uqSut,10.129.248.21,60344,192.168.220.73,993,tcp,ssl,1.925477,14345.0,1.882681e+06,SF,True,True,0,ShADadtfFRR,187,24209,1372,1956929,2024-02-17 16:52:00.255445003,2024-02-17 16:52:00
24180,1.708189e+09,CRqRb24rQapqmqJ1Ih,10.129.248.21,56010,192.168.220.73,993,tcp,ssl,3.247989,14340.0,1.881264e+06,SF,True,True,0,ShADadtWWtfFRR,215,25684,257,1905558,2024-02-17 16:51:58.960154057,2024-02-17 16:51:00
24222,1.708189e+09,CBMTEvj0dPcLdas7b,10.158.115.188,52786,10.100.123.73,993,tcp,ssl,1.886200,14345.0,1.879826e+06,SF,True,True,25141,ShADadgtfGFRR,246,27353,1379,1962823,2024-02-17 16:52:58.934120893,2024-02-17 16:52:00
24247,1.708188e+09,COCwi43lvG3NbtWy65,10.128.0.211,33264,10.128.0.202,22,tcp,NaN,1091.179739,867408.0,2.162000e+06,OTH,True,True,0,DdAaT,21984,2010720,21724,3291696,2024-02-17 16:37:49.172693968,2024-02-17 16:37:00


In [36]:
get_counts(large_re_b, "id.resp_h")

,id.resp_h,Count
26,192.168.220.73,52
7,10.100.108.73,6
10,10.100.111.73,6
16,10.100.117.73,6
1,10.100.102.73,5
2,10.100.103.73,5
4,10.100.105.73,5
6,10.100.107.73,5
5,10.100.106.73,5
3,10.100.104.73,5


In [37]:
hist(large_re_b, "resp_bytes")

# dns.log

In [38]:
dns_df

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,trans_id,rtt,query,qclass,qclass_name,qtype,qtype_name,rcode,rcode_name,AA,TC,RD,RA,Z,answers,TTLs,rejected,dt
0,1.708188e+09,CPhVJq1LgdVdTF94a5,192.168.220.53,58699,192.168.220.132,53,udp,19068,0.000625,pasta.wobblys.world,1.0,C_INTERNET,1.0,A,0.0,NOERROR,True,False,True,True,0,[192.168.220.132],[3600.0],False,2024-02-17 16:37:17.133541106
1,1.708188e+09,CPV3MtHunsW4BYbWd,192.168.220.53,49593,192.168.220.132,53,udp,54674,NaN,pasta.wobblys.world,1.0,C_INTERNET,28.0,AAAA,0.0,NOERROR,False,False,True,False,0,NaN,NaN,False,2024-02-17 16:37:17.133546114
2,1.708188e+09,CHLwCOUHhBK0B6XDf,192.168.220.241,62361,192.168.220.132,53,udp,52790,NaN,wpad.wobblys.world,1.0,C_INTERNET,1.0,A,3.0,NXDOMAIN,False,False,True,False,0,NaN,NaN,False,2024-02-17 16:37:18.664900064
3,1.708188e+09,CB09Nd2zEhpyXchdP9,192.168.220.132,60370,10.0.0.2,53,udp,13340,NaN,ext-services-lb-2a685576fdf664e2.elb.us-west-2...,1.0,C_INTERNET,28.0,AAAA,0.0,NOERROR,False,False,True,False,0,NaN,NaN,False,2024-02-17 16:37:19.866745949
4,1.708188e+09,CWea114iwJwcF2HyO1,192.168.220.43,31731,192.168.220.132,53,udp,57040,0.002800,git.rancher.io,1.0,C_INTERNET,28.0,AAAA,0.0,NOERROR,False,False,True,True,0,[ext-services-lb-2a685576fdf664e2.elb.us-west-...,[125.0],False,2024-02-17 16:37:19.866159916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4748,1.708189e+09,C1qQXJdKkLUgtfg13,10.179.165.24,5353,224.0.0.251,5353,udp,0,NaN,debian.local,1.0,C_INTERNET,255.0,*,NaN,NaN,False,False,False,False,0,NaN,NaN,False,2024-02-17 16:55:59.829093933
4749,1.708189e+09,C1qQXJdKkLUgtfg13,10.179.165.24,5353,224.0.0.251,5353,udp,0,NaN,debian.local,1.0,C_INTERNET,255.0,*,NaN,NaN,False,False,False,False,0,NaN,NaN,False,2024-02-17 16:55:59.939397097
4750,1.708189e+09,C1qQXJdKkLUgtfg13,10.179.165.24,5353,224.0.0.251,5353,udp,0,NaN,debian.local,1.0,C_INTERNET,255.0,*,NaN,NaN,False,False,False,False,0,NaN,NaN,False,2024-02-17 16:55:59.939404011
4751,1.708189e+09,C1qQXJdKkLUgtfg13,10.179.165.24,5353,224.0.0.251,5353,udp,0,NaN,debian.local,1.0,C_INTERNET,255.0,*,NaN,NaN,False,False,False,False,0,NaN,NaN,False,2024-02-17 16:56:00.079137087


In [39]:
# check for resp_h out of known dns servers

In [40]:
get_counts(dns_df, "id.orig_h")

,id.orig_h,Count
16,192.168.220.111,697
23,192.168.220.66,653
26,192.168.220.84,526
17,192.168.220.132,452
18,192.168.220.2,367
20,192.168.220.30,271
19,192.168.220.241,196
21,192.168.220.43,186
7,10.129.248.21,183
9,10.158.115.188,171


In [41]:
get_counts(dns_df, "id.orig_h", time_slot="1min").head(20)

,id.orig_h,time_slot,Count
46,10.166.245.91,2024-02-17 16:49:00,161
56,192.168.220.111,2024-02-17 16:40:00,133
39,10.129.248.21,2024-02-17 16:52:00,119
42,10.158.115.188,2024-02-17 16:53:00,106
190,192.168.220.66,2024-02-17 16:52:00,102
45,10.162.244.85,2024-02-17 16:54:00,100
187,192.168.220.66,2024-02-17 16:49:00,93
191,192.168.220.66,2024-02-17 16:53:00,86
59,192.168.220.111,2024-02-17 16:43:00,83
66,192.168.220.111,2024-02-17 16:50:00,79


In [42]:
long_queries = dns_df[dns_df["query"].str.len() > 30]
get_counts(long_queries, "id.orig_h")

,id.orig_h,Count
3,192.168.220.132,154
7,192.168.220.69,53
2,192.168.220.111,48
8,192.168.220.84,32
5,192.168.220.30,19
1,10.100.117.2,17
0,10.100.116.2,16
6,192.168.220.53,10
4,192.168.220.241,5


In [43]:
get_counts(long_queries, "id.orig_h", time_slot="1min")

/Users/nic/projects/zeek_playground/utils/aggregate_logs.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id.orig_h,time_slot,Count
38,192.168.220.132,2024-02-17 16:54:00,15
23,192.168.220.132,2024-02-17 16:39:00,14
28,192.168.220.132,2024-02-17 16:44:00,14
33,192.168.220.132,2024-02-17 16:49:00,14
19,192.168.220.111,2024-02-17 16:50:00,12
...,...,...,...
61,192.168.220.53,2024-02-17 16:52:00,1
58,192.168.220.53,2024-02-17 16:45:00,1
55,192.168.220.53,2024-02-17 16:41:00,1
56,192.168.220.53,2024-02-17 16:42:00,1


In [44]:
get_counts(dns_df, "query")

,query,Count
48,pasta.wobblys.world,995
47,orzo.wobblys.world,534
33,carbonara.local,335
44,git.rancher.io,285
49,radiatori.wobblys.world,236
...,...,...
62,registry.wobblys9.world,2
71,win-global-asimov-leafs-events-data.trafficman...,2
66,time1.vyos.net,2
63,settings-prod-eus2-2.eastus2.cloudapp.azure.com,1


In [45]:
get_counts(dns_df, ["id.orig_h", "query"], time_slot="1min")

,id.orig_h,query,time_slot,Count
114,10.166.245.91,pasta.wobblys.world,2024-02-17 16:49:00,149
97,10.129.248.21,pasta.wobblys.world,2024-02-17 16:52:00,105
104,10.158.115.188,pasta.wobblys.world,2024-02-17 16:53:00,104
111,10.162.244.85,pasta.wobblys.world,2024-02-17 16:54:00,96
538,192.168.220.66,orzo.wobblys.world,2024-02-17 16:49:00,88
...,...,...,...,...
15,10.100.116.2,21.248.129.10.in-addr.arpa,2024-02-17 16:51:00,1
14,10.100.116.2,188.115.158.10.in-addr.arpa,2024-02-17 16:52:00,1
13,10.100.116.2,161.36.159.10.in-addr.arpa,2024-02-17 16:50:00,1
12,10.100.116.2,140.105.180.10.in-addr.arpa,2024-02-17 16:55:00,1


In [46]:
dns_df["query_entropy"] = dns_df["query"].apply(compute_entropy)

In [47]:
hist(dns_df, "query_entropy")

In [48]:
_df = get_summary_stats(dns_df, "id.orig_h", "query_entropy", time_slot="1min")
_df[_df["conn_count"] > 5].head(20)

,id.orig_h,time_slot,query_entropy_mean,query_entropy_std,conn_count
72,192.168.220.132,2024-02-17 16:37:00,4.226289,0.430495,13
203,192.168.220.69,2024-02-17 16:48:00,4.200275,0.393930,6
199,192.168.220.69,2024-02-17 16:43:00,4.127127,0.381315,11
89,192.168.220.132,2024-02-17 16:54:00,4.113848,0.492222,23
76,192.168.220.132,2024-02-17 16:41:00,4.100151,0.486330,12
81,192.168.220.132,2024-02-17 16:46:00,4.074337,0.578808,12
73,192.168.220.132,2024-02-17 16:38:00,4.049765,0.537247,9
77,192.168.220.132,2024-02-17 16:42:00,3.980801,0.577660,18
161,192.168.220.53,2024-02-17 16:37:00,3.922838,0.298605,8
83,192.168.220.132,2024-02-17 16:48:00,3.921712,0.382772,15


In [49]:
high_entropy_queries = dns_df[dns_df["query_entropy"] > 4]

In [50]:
get_counts(high_entropy_queries, "id.orig_h", time_slot="1min")

/Users/nic/projects/zeek_playground/utils/aggregate_logs.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id.orig_h,time_slot,Count
34,192.168.220.132,2024-02-17 16:49:00,14
24,192.168.220.132,2024-02-17 16:39:00,14
39,192.168.220.132,2024-02-17 16:54:00,14
21,192.168.220.111,2024-02-17 16:55:00,12
18,192.168.220.111,2024-02-17 16:40:00,12
...,...,...,...
57,192.168.220.53,2024-02-17 16:51:00,1
55,192.168.220.53,2024-02-17 16:45:00,1
54,192.168.220.53,2024-02-17 16:44:00,1
52,192.168.220.53,2024-02-17 16:41:00,1


In [51]:
get_counts(dns_df, "qtype_name")

,qtype_name,Count
1,A,2181
2,AAAA,977
7,PTR,466
9,SRV,262
5,NIMLOC,248
6,NS,176
0,*,108
3,DS,68
8,SOA,11
4,MX,5


In [52]:
get_counts(dns_df, "rcode_name")

,rcode_name,Count
0,NOERROR,2665
1,NXDOMAIN,662


In [53]:
dns_errors = dns_df[dns_df["rcode_name"] == "NXDOMAIN"]
get_counts(dns_errors, "id.orig_h")

,id.orig_h,Count
2,192.168.220.111,133
8,192.168.220.84,130
3,192.168.220.132,123
7,192.168.220.73,122
6,192.168.220.69,88
4,192.168.220.241,26
1,10.100.117.2,22
0,10.100.116.2,17
5,192.168.220.30,1


In [54]:
get_counts(dns_errors, "id.orig_h", time_slot="1min")

/Users/nic/projects/zeek_playground/utils/aggregate_logs.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id.orig_h,time_slot,Count
99,192.168.220.84,2024-02-17 16:49:00,27
25,192.168.220.111,2024-02-17 16:40:00,26
29,192.168.220.111,2024-02-17 16:45:00,24
33,192.168.220.111,2024-02-17 16:50:00,24
45,192.168.220.132,2024-02-17 16:49:00,24
...,...,...,...
44,192.168.220.132,2024-02-17 16:46:00,1
57,192.168.220.241,2024-02-17 16:43:00,1
58,192.168.220.241,2024-02-17 16:44:00,1
82,192.168.220.69,2024-02-17 16:51:00,1


# http.log

**Cols of interest:**
* Origin port & IP
* Destination port & IP
* Method
* host (domain)
* Uri (query, resource being requested)
* user agent
* referrer (url that redirected there)
* status code
* resp_mime_type

**Triage:**
* Check for excessive POST requests from same internal IP (Data exfiltration)
* Check for very long response body lenght, with destination IP being internal (data exfiltration)
* Check for unusual URIs (SQL injection)
    1. Check for queries containing -- ; = "
    2. Check for unusual query patterns
* Check for unusual user_agent (indicating bot traffic)
* Check for fake login pages in URI (phising)
* Check for cross-site scripting
    1. check for `<script>` in URI
* Check for brute force attempts
    1. Many POST to login page from same IP address
    2. Many 400X error
* Check for malware download
    1. 

# http.log



In [55]:
http_df

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,trans_depth,method,host,uri,version,user_agent,request_body_len,response_body_len,status_code,status_msg,tags,resp_fuids,resp_mime_types,orig_fuids,orig_mime_types,referrer,resp_filenames,origin,proxied,dt
0,1.708188e+09,CPUUms2KFKLkmd6Mz,192.168.220.111,58192,68.142.107.4,80,1,GET,ctldl.windowsupdate.com,/msdownload/update/v3/static/trustedr/en/authr...,1.1,Microsoft-CryptoAPI/10.0,0,0,304.0,Not Modified,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-17 16:37:30.818512917
1,1.708188e+09,CGfWld1IbegDVRRZZf,10.100.116.2,50564,68.142.107.4,80,1,GET,ctldl.windowsupdate.com,/msdownload/update/v3/static/trustedr/en/disal...,1.1,Microsoft-CryptoAPI/10.0,0,0,304.0,Not Modified,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-17 16:37:41.496485949
2,1.708188e+09,CvcUrA3u8uuh7PIe87,192.168.220.69,50564,68.142.107.4,80,1,GET,ctldl.windowsupdate.com,/msdownload/update/v3/static/trustedr/en/disal...,1.1,Microsoft-CryptoAPI/10.0,0,0,304.0,Not Modified,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-17 16:37:41.496479034
3,1.708188e+09,CjLzl6WmlTPKnBaA8,192.168.220.132,59517,68.142.107.4,80,1,GET,ctldl.windowsupdate.com,/msdownload/update/v3/static/trustedr/en/authr...,1.1,Microsoft-CryptoAPI/10.0,0,0,304.0,Not Modified,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-17 16:37:52.456943035
4,1.708188e+09,CSqzgB2mvVtdGKKGpg,192.168.220.132,62114,68.142.107.4,80,1,GET,ctldl.windowsupdate.com,/msdownload/update/v3/static/trustedr/en/disal...,1.1,Microsoft-CryptoAPI/10.0,0,0,304.0,Not Modified,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-17 16:39:19.145517111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3953,1.708189e+09,CVuqSn6xkomKW4WDe,10.179.165.24,36608,10.100.121.121,80,1,GET,bucatini.wobblys.world,/dokuwiki,NaN,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,0,0,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-17 16:56:00.161715984
3954,1.708189e+09,C0b3T02HlXlBdpgTDe,10.179.165.24,49180,10.100.114.121,80,1,GET,bucatini.wobblys.world,/dokuwiki/doku.php?id=start,NaN,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,0,0,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-17 16:56:00.118706942
3955,1.708189e+09,C4rTf44TSYn5TmBLfk,10.179.165.24,34264,10.100.104.73,80,1,GET,dabigtortellini.wobblys.world,/,NaN,Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:61...,0,0,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-17 16:56:00.142997980
3956,1.708189e+09,Cleon51yaIW3E0MlI,192.168.220.132,51590,192.168.220.111,80,1,CCM_POST,gnocchi.wobblys.world,/ccm_system/request,1.1,ccmhttp,0,92,200.0,OK,[],[FV91PP1zMNeWWUJnr3],NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-17 16:55:00.323760033


In [56]:
get_counts(http_df, "id.orig_h")

,id.orig_h,Count
11,10.162.244.85,477
12,10.166.245.91,471
9,10.158.115.188,470
14,10.180.105.140,465
7,10.129.248.21,461
6,10.128.177.91,456
8,10.135.237.219,454
10,10.159.36.161,291
3,10.128.0.26,119
13,10.179.165.24,109


In [57]:
get_counts(http_df, "method")

,method,Count
1,GET,3359
3,POST,409
2,OPTIONS,20
0,CCM_POST,2


In [58]:
post_requests = http_df[http_df["method"] == "POST"]
get_counts(post_requests, "id.orig_h")

,id.orig_h,Count
3,10.129.248.21,59
6,10.162.244.85,59
5,10.158.115.188,59
8,10.180.105.140,58
4,10.135.237.219,58
2,10.128.177.91,56
7,10.166.245.91,53
1,10.128.1.95,6
0,10.128.0.209,1


In [59]:
non_empty_uri = http_df[~http_df.uri.isna()]

In [60]:
risky_str = ["/login", "/admin"]
risky_uri = non_empty_uri[non_empty_uri["uri"].str.contains('|'.join(risky_str))]
risky_uri

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,trans_depth,method,host,uri,version,user_agent,request_body_len,response_body_len,status_code,status_msg,tags,resp_fuids,resp_mime_types,orig_fuids,orig_mime_types,referrer,resp_filenames,origin,proxied,dt
10,1.708188e+09,CRvLtX1cZ2Sl7Bnno8,10.128.177.91,46244,10.100.118.73,80,1,GET,dabigtortellini.wobblys.world,/src/login.php,1.1,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,0,2880,200.0,OK,[],[FnfHQF2IEzGZlqnXng],[text/html],NaN,NaN,NaN,NaN,NaN,NaN,2024-02-17 16:40:30.642263889
21,1.708188e+09,C5WARl1ngxXikCMt7h,10.128.177.91,35580,192.168.220.73,80,1,GET,dabigtortellini.wobblys.world,/src/login.php,1.1,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,0,2880,200.0,OK,[],[Fw7Cyc4VxZGuwuxmJf],[text/html],NaN,NaN,NaN,NaN,NaN,NaN,2024-02-17 16:40:30.676019907
22,1.708188e+09,Cp1yUG2Zc207wf57G1,10.128.177.91,35580,10.100.105.73,80,1,GET,dabigtortellini.wobblys.world,/src/login.php,1.1,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,0,2880,200.0,OK,[],[F9WVoD28MJGkKwg0Ya],[text/html],NaN,NaN,NaN,NaN,NaN,NaN,2024-02-17 16:40:30.675381899
28,1.708188e+09,Cv5vJE3vO4wFBWDlc,10.128.177.91,41008,10.100.114.73,80,1,GET,dabigtortellini.wobblys.world,/src/login.php,1.1,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,0,2880,200.0,OK,[],[F8EeCa3ouQLjZOPiSg],[text/html],NaN,NaN,NaN,NaN,NaN,NaN,2024-02-17 16:40:30.698256016
44,1.708188e+09,CM1kOw2yOGErDeOJ87,10.128.177.91,58830,10.100.103.73,80,1,GET,dabigtortellini.wobblys.world,/src/login.php,1.1,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,0,2880,200.0,OK,[],[FvWseyO7v0ykBtXT8],[text/html],NaN,NaN,NaN,NaN,NaN,NaN,2024-02-17 16:40:30.789885998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3946,1.708189e+09,CtfJbH2lCvOev5NxH8,10.179.165.24,53268,192.168.220.30,80,1,GET,orzo.wobblys.world,/auth/login,NaN,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:61...,0,0,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-17 16:56:00.313198090
3949,1.708189e+09,CX2376a3k4cq4Hzqh,10.179.165.24,59144,10.100.105.30,80,1,GET,orzo.wobblys.world,/auth/login,NaN,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:61...,0,0,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-17 16:56:00.275623083
3950,1.708189e+09,CcKsYI3dj8lS1rtXF7,10.179.165.24,58978,192.168.220.30,80,1,GET,orzo.wobblys.world,/auth/login,NaN,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:61...,0,0,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-17 16:56:00.261723042
3951,1.708189e+09,CzRqJf3OCCiqQ3pzVk,10.179.165.24,58978,10.100.102.30,80,1,GET,orzo.wobblys.world,/auth/login,NaN,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:61...,0,0,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-17 16:56:00.261199951


In [61]:
get_counts(risky_uri, "id.orig_h")

,id.orig_h,Count
8,10.162.244.85,68
6,10.158.115.188,67
3,10.128.177.91,66
11,10.180.105.140,66
5,10.135.237.219,66
9,10.166.245.91,64
7,10.159.36.161,64
4,10.129.248.21,63
10,10.179.165.24,28
1,10.128.0.26,27


In [62]:
fishy_str = [";", "--", "INSERT", "DELETE"]
fishy_uri = non_empty_uri[non_empty_uri["uri"].str.contains('|'.join(fishy_str))]
fishy_uri

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,trans_depth,method,host,uri,version,user_agent,request_body_len,response_body_len,status_code,status_msg,tags,resp_fuids,resp_mime_types,orig_fuids,orig_mime_types,referrer,resp_filenames,origin,proxied,dt


In [63]:
get_counts(http_df, "status_code")

,status_code,Count
1,200.0,2629
3,302.0,672
2,301.0,251
6,400.0,62
5,304.0,25
0,0.0,10
4,303.0,3
7,404.0,3
8,500.0,1
9,503.0,1


In [64]:
get_counts(http_df, "user_agent")

,user_agent,Count
25,python-requests/2.21.0,1230
22,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,309
10,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,210
13,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:61...,189
3,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,166
4,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,159
12,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,155
11,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,151
20,Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:66...,141
18,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,135


In [65]:
hist(http_df, "request_body_len")

In [66]:
large_request = http_df[http_df["request_body_len"] > 100]
get_counts(large_request, ["id.orig_h", "method"])

,id.orig_h,method,Count
2,10.129.248.21,POST,59
5,10.162.244.85,POST,59
4,10.158.115.188,POST,59
7,10.180.105.140,POST,58
3,10.135.237.219,POST,58
1,10.128.177.91,POST,56
6,10.166.245.91,POST,53
0,10.128.1.95,POST,3
8,192.168.220.132,CCM_POST,1


In [67]:
hist(http_df, "response_body_len")

In [68]:
hist(http_df[http_df["response_body_len"] < 50000], "response_body_len")

In [69]:
large_response = http_df[http_df["response_body_len"] > 40000]
get_counts(large_response, ["id.orig_h", "method"])

,id.orig_h,method,Count
9,10.162.244.85,GET,31
4,10.128.177.91,GET,30
7,10.158.115.188,GET,29
12,10.180.105.140,GET,28
6,10.135.237.219,GET,27
8,10.159.36.161,GET,27
5,10.129.248.21,GET,26
10,10.166.245.91,GET,25
1,10.128.0.26,GET,13
2,10.128.1.95,GET,11


In [70]:
get_counts(http_df.explode("resp_mime_types"), ["method", "resp_mime_types"])

,method,resp_mime_types,Count
3,GET,text/html,2368
8,POST,text/html,374
1,GET,image/gif,207
5,GET,text/plain,34
4,GET,text/json,10
7,OPTIONS,text/plain,9
6,OPTIONS,text/html,2
0,GET,application/font-woff2,1
2,GET,image/x-icon,1
9,POST,text/json,1


In [71]:
get_counts(post_requests.explode("orig_mime_types"), "orig_mime_types")

,orig_mime_types,Count
1,text/plain,408
0,text/json,1


---

In [72]:
# Status code counts
status_code_counts = http_df['status_code'].value_counts().reset_index()
status_code_counts.columns = ["Status Code", "Count"]
status_code_counts["Status Code"] = status_code_counts["Status Code"].astype(str)

# Bar chart
fig = px.bar(status_code_counts, x="Status Code", y="Count", title="HTTP Status Code Distribution",
             labels={"Count": "Number of Responses", "Status Code": "HTTP Status Code"}, template="plotly_dark")
fig.update_layout(xaxis_title="HTTP Status Code", yaxis_title="Number of Responses")
fig.show()


In [73]:
# Status code counts
status_code_counts = http_df['status_code'].value_counts().reset_index()
status_code_counts.columns = ["Status Code", "Count"]
status_code_counts = status_code_counts[status_code_counts["Status Code"] > 399]
status_code_counts["Status Code"] = status_code_counts["Status Code"].astype(str)

# Bar chart
fig = px.bar(status_code_counts, x="Status Code", y="Count", title="HTTP Status Code Distribution",
             labels={"Count": "Number of Responses", "Status Code": "HTTP Status Code"}, template="plotly_dark")
fig.update_layout(xaxis_title="HTTP Status Code", yaxis_title="Number of Responses")
fig.show()


In [74]:
# Plot histogram of response body lengths
fig = px.histogram(http_df, x="response_body_len", nbins=50, title="Histogram of HTTP Response Body Lengths",
                   labels={"response_body_len": "Response Body Length (bytes)"}, template="plotly_dark")
fig.update_layout(xaxis_title="Response Body Length (bytes)", yaxis_title="Frequency")
fig.show()

# Top 10 largest responses
top_responses = http_df.nlargest(10, 'response_body_len')
fig = px.bar(top_responses, x="uri", y="response_body_len",
             hover_data=["id.orig_h", "host"], title="Top 10 Largest HTTP Responses",
             labels={"uri": "URI", "response_body_len": "Response Body Length (bytes)"}, template="plotly_dark")
fig.update_layout(xaxis_title="URI", yaxis_title="Response Body Length (bytes)")
fig.show()


# TS

In [84]:
ts_df = get_ts(conn_df, [], "id.orig_h", time_slot="1min", mode="count")

In [85]:
ts_line(ts_df, "count")

In [87]:
ts_line(ts_df, "count", "id.orig_h")

In [88]:
ts_df = get_ts(conn_df, ["id.orig_h"], "orig_bytes", time_slot="1min", mode="sum")

In [89]:
ts_line(ts_df, "sum")

In [90]:
ts_line(ts_df, "sum", "id.orig_h")

In [91]:
ts_df = get_ts(conn_df, ["id.orig_h"], "duration", time_slot="1min", mode="mean")

In [92]:
ts_line(ts_df, "mean")

In [93]:
ts_line(ts_df, "mean", "id.orig_h")